# Project 3:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [748]:
# PREPARE REQUIRED LIBRARIES
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
import requests
import urllib
# import urllib.requests
import pandas as pd
from pandas import Series, DataFrame
from bs4 import BeautifulSoup



## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

In [749]:
# SCRAPE WIKIPEDI<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: A FOR LARGEST US CITIES (NON-EXHAUSTIVE LIST)


In [750]:
large_cities = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'


In [751]:
response = requests.get(large_cities)
HTML = response.text
HTML[:250]

u'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of United States cities by population - Wikipedia</title>\n<script>document.documentElement.className = document.documentElement.className.replac'

In [752]:
lista_US = Selector(text=HTML).xpath('//td[2]/i/a/text()').extract()


In [831]:
lista_us_pd = pd.DataFrame(lista_US)

In [834]:
type(lista_US)

list

In [835]:
lista_us_pd.to_csv('lista_us_cities.csv', index=False)

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

In [753]:
a = '(danville / san ramon)'

In [754]:
a[1:-1]

'danville / san ramon'

In [755]:
lista_US

[u'New York',
 u'Los Angeles',
 u'Chicago',
 u'Houston',
 u'Philadelphia',
 u'Jacksonville',
 u'Charlotte',
 u'Seattle',
 u'Detroit',
 u'Portland',
 u'Las Vegas',
 u'Louisville',
 u'Baltimore',
 u'Milwaukee',
 u'Albuquerque',
 u'Kansas City',
 u'Virginia Beach',
 u'Omaha',
 u'Minneapolis',
 u'New Orleans',
 u'Wichita',
 u'Anchorage',
 u'Newark',
 u'Birmingham',
 u'Sioux Falls',
 u'Bridgeport',
 u'Charleston',
 u'Fargo',
 u'Manchester',
 u'Billings']

## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [756]:
url = 'https://sfbay.craigslist.org/search/sss?sort=rel.html'

In [757]:
response = requests.get(url)
HTML_sf = response.text
HTML_sf[:150]

u'\ufeff<!DOCTYPE html>\n\n<html class="no-js"><head>\n    <title>SF bay area for sale  - craigslist</title>\n\n    <meta name="description" content="SF bay area '

In [758]:
def sf_scraping(html_sf):
    lista_price = [price[1:] for price in Selector(text=html_sf).xpath('//span[@class]/span[@class="result-price"]//text()').extract()]
    return lista_price


In [759]:
print sf_scraping(HTML_sf)

[u'500', u'75', u'150', u'75', u'130000', u'25', u'10', u'695', u'17999', u'25', u'120', u'200', u'50', u'250', u'500', u'25', u'19999', u'80', u'150', u'6900', u'40', u'60', u'390', u'1444', u'8', u'10', u'40', u'390', u'450', u'10', u'10', u'18500', u'50', u'5', u'200', u'50', u'40000', u'475', u'79', u'10', u'50', u'4800', u'125', u'35', u'240', u'7500', u'1700', u'1500', u'26000', u'20', u'5', u'10950', u'7', u'2000', u'370', u'75', u'10', u'100', u'7800', u'5', u'100', u'6', u'250', u'12', u'7', u'125', u'100', u'4700', u'950', u'15', u'180', u'200', u'675', u'100', u'150', u'14888', u'4', u'180', u'3000', u'125', u'7300', u'50', u'165', u'25', u'30', u'50', u'1095', u'12', u'400', u'3', u'1', u'300', u'60', u'8900', u'400', u'9499', u'16999', u'12800', u'2937', u'3500']



## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [760]:
import numpy as np

In [761]:
# response_rv = requests.get('https://chicago.craigslist.org/search/sss?query=rv')

# HTML_rv = response_rv.text


In [762]:
# Selector(text=HTML_rv).xpath('//ul[@class="rows"]/li[@class="result-row"]/p[@class="result-info"]')

In [763]:
# city_prices = Selector(text=HTML_rv).xpath('//span[@class]')
# titles = Selector(text=HTML_rv).xpath('//a[@class="result-title hdrlnk"]//text()')

In [764]:
# HTML_rv

In [765]:
# def extract_info(html):
#     lista_price = [join(price[1:]) for price in Selector(text=html).xpath('//span[@class]/span[@class="result-price"]//text()').extract()]
#     # lista_price
# #     lista_city = [' '.join(city[2:-1]) for city in Selector(text=html).xpath('//span[@class="result-hood"]//text()').extract()]
#     # lista_city
# #     lista_title = Selector(text=html).xpath('//a[@class="result-title hdrlnk"]//text()').extract()
#     # lista_title
#     return lista_price


In [766]:
city_dict = {}
for us in lista_US:
    url_rv = "http://{}.craigslist.org/search/sss?query=rv".format(us)
    try:
        response_rv = requests.get(url_rv)
        if response_rv.status_code != 200:
            continue
    except:
        continue
    HTML_rv = response_rv.content
    lista_price = [price[1:] for price in Selector(text=HTML_rv).xpath('//span[@class]/span[@class="result-price"]//text()').extract()]
    lista_price = map(lambda x: int(x), lista_price)
    if len(lista_price) < 120:
        for _ in range((120 - len(lista_price))):
#             lista_price.append(round(np.mean(lista_price), 2))
            lista_price.append(0)

    city_dict[us] = lista_price

In [767]:
city_prices = pd.DataFrame(city_dict)

In [768]:
for city in city_prices:
    if city_prices[city].isnull().any():
        city_prices.drop(labels=city, axis=1, inplace=True)

In [769]:
city_prices = city_prices[~(city_prices == 0).all(axis=1)]

In [770]:
city_prices.isnull().any().any()

False

In [771]:
# next_ = Selector(text=HTML_rv).xpath('//span/a[@class="button next"]/@href').extract()[0]


## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

In [772]:
def calculate_mean(col):
    return col.mean()

def calculate_median(col):
    return col.median()

In [773]:
cp_mean_median = {}
for city in city_prices:
    cp_mean_median[city] = [round(calculate_mean(city_prices[city]), 2), 
                            calculate_median(city_prices[city])]

In [774]:
cityprice_med = pd.DataFrame(cp_mean_median).T
cityprice_med.rename(columns={0: 'Mean', 1: 'Median'}, inplace=True)
cityprice_med = cityprice_med.T
cityprice_med

,Albuquerque,Anchorage,Baltimore,Billings,Birmingham,Charleston,Charlotte,Chicago,Detroit,Fargo,...,Jacksonville,Louisville,Manchester,Milwaukee,Minneapolis,Omaha,Philadelphia,Portland,Seattle,Wichita
Mean,13943.19,12730.79,9802.55,19869.91,0.0,10475.6,12268.25,8870.52,9269.8,17930.47,...,19436.92,16628.26,2.59,16246.14,20730.55,19565.51,11184.77,16696.93,6970.39,10378.22
Median,6000.00,290.00,1099.00,15995.00,0.0,8000.0,795.00,90.00,400.0,16195.00,...,1000.00,10000.00,0.00,6500.00,19556.00,17995.00,3376.00,700.00,50.00,1290.00


In [775]:
city_prices.shape, cityprice_med.shape

((113, 21), (2, 21))

In [776]:
city_prices = city_prices.append(cityprice_med)

In [777]:
city_prices

,Albuquerque,Anchorage,Baltimore,Billings,Birmingham,Charleston,Charlotte,Chicago,Detroit,Fargo,...,Jacksonville,Louisville,Manchester,Milwaukee,Minneapolis,Omaha,Philadelphia,Portland,Seattle,Wichita
0,29000.00,200.00,4000.00,15995.00,0.0,35.0,89.00,60.00,35000.0,38816.00,...,1.00,215.00,103.00,11600.00,28999.00,28999.00,24995.00,13500.00,19988.00,15.00
1,3500.00,200.00,95.00,15995.00,0.0,500.0,24995.00,12500.00,9300.0,36924.00,...,149.00,299.00,190.00,9900.00,44599.00,35995.00,9000.00,26999.00,50.00,21999.00
2,29000.00,39900.00,65.00,15995.00,0.0,11900.0,275.00,18000.00,53000.0,27988.00,...,149.00,9500.00,0.00,8000.00,28999.00,33599.00,35.00,29999.00,27000.00,3000.00
3,15500.00,125.00,90.00,17500.00,0.0,24995.0,49500.00,17900.00,5800.0,69649.00,...,29000.00,19500.00,0.00,19750.00,24999.00,30999.00,14000.00,38900.00,44995.00,450.00
4,28900.00,175.00,20000.00,15995.00,0.0,8900.0,11900.00,40.00,5800.0,69649.00,...,29000.00,150.00,0.00,650.00,43999.00,32999.00,22900.00,34999.00,250.00,9500.00
5,8900.00,250.00,1000.00,14500.00,0.0,8500.0,200.00,4900.00,40.0,15600.00,...,57977.00,3000.00,0.00,49900.00,41877.00,34999.00,1995.00,119900.00,19950.00,1000.00
6,1550.00,60.00,25.00,2000.00,0.0,5500.0,225.00,72500.00,7000.0,24999.00,...,18500.00,18000.00,0.00,11500.00,38816.00,33999.00,250.00,35000.00,15995.00,10.00
7,1.00,3500.00,1900.00,15995.00,0.0,7500.0,37870.00,50.00,6950.0,53500.00,...,28900.00,10500.00,0.00,1.00,36924.00,36995.00,17585.00,119999.00,23980.00,15.00
8,329.00,100.00,250.00,250.00,0.0,9900.0,1.00,12995.00,35.0,12995.00,...,4000.00,24500.00,0.00,40.00,27988.00,26599.00,50.00,21500.00,20000.00,150.00
9,125.00,10.00,24500.00,15995.00,0.0,6500.0,9000.00,16000.00,4000.0,1234.00,...,150.00,800.00,0.00,25000.00,69649.00,20999.00,50.00,17995.00,57825.00,50.00



## 4. Run your scraping process, and save your results to a CSV file.

In [778]:
city_prices.to_csv('city_prices_rv.csv')


## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?

In [779]:
city_prices.tail(2).T.sort_values(['Mean'], ascending=False)

,Mean,Median
Houston,20870.83,0.0
Minneapolis,20730.55,19556.0
Billings,19869.91,15995.0
Omaha,19565.51,17995.0
Jacksonville,19436.92,1000.0
Fargo,17930.47,16195.0
Portland,16696.93,700.0
Louisville,16628.26,10000.0
Milwaukee,16246.14,6500.0
Albuquerque,13943.19,6000.0


In [780]:
# city_prices.T.groupby(['Mean'], sort=False).sum()

#### A: Only few cities with no meaningful data. Houston seems to be the city with the most revenue, but the mean is 0, which makes me ignore the market. I'll pick the second, which is Omaha


### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

#### A: If the idea is such, I'd buy in Minneapolis and sell in Omaha, with a revenue of around 3k

In [791]:
city_prices.tail(2).T.sort_values(['Mean'], ascending=False).head()

,Mean,Median
Houston,20870.83,0.0
Minneapolis,20730.55,19556.0
Billings,19869.91,15995.0
Omaha,19565.51,17995.0
Jacksonville,19436.92,1000.0


### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

In [804]:
make_dict = {}
for us in lista_US:
    url_make = "http://{}.craigslist.org/search/sss?query=rv&auto_make_model=winnebago".format(us)
    try:
        response_make = requests.get(url_make)
        if response_make.status_code != 200:
            continue
    except:
        continue
    HTML_make = response_make.content
    lista_make = [make[1:] for make in Selector(text=HTML_make).xpath('//span[@class]/span[@class="result-price"]//text()').extract()]
    lista_make = map(lambda x: int(x), lista_make)
    if len(lista_make) < 120:
        for _ in range((120 - len(lista_make))):
            lista_make.append(0)

    make_dict[us] = lista_make

In [811]:
make_prices = pd.DataFrame(make_dict)

In [812]:
make_prices.isnull().any().any()

False

In [813]:
make_prices = make_prices[~(make_prices == 0).all(axis=1)]

In [817]:
# make_prices

In [818]:
make_mean_median = {}
for make in make_prices:
    make_mean_median[make] = [round(calculate_mean(make_prices[make]), 2), 
                            calculate_median(make_prices[make])]

In [820]:
# make_mean_median

In [821]:
citymake_med = pd.DataFrame(make_mean_median).T
citymake_med.rename(columns={0: 'Mean', 1: 'Median'}, inplace=True)
citymake_med = citymake_med.T
citymake_med

,Albuquerque,Anchorage,Baltimore,Billings,Birmingham,Charleston,Charlotte,Chicago,Detroit,Fargo,...,Jacksonville,Louisville,Manchester,Milwaukee,Minneapolis,Omaha,Philadelphia,Portland,Seattle,Wichita
Mean,100564.9,30145.76,17790.64,16964.9,0.0,15186.23,21001.05,11989.18,10690.53,10651.36,...,24278.28,7247.91,0.0,15020.63,30884.73,13466.4,22037.41,17227.08,37128.44,16436.47
Median,0.0,29000.00,0.00,2000.0,0.0,0.00,6000.00,0.00,0.00,0.00,...,16977.00,0.00,0.0,1200.00,17000.00,0.0,100.00,0.00,27999.00,0.00


In [822]:
city_make = make_prices.append(citymake_med)

In [824]:
# city_make

In [825]:
city_prices.tail(2).T.sort_values(['Mean'], ascending=False).head()

,Mean,Median
Houston,20870.83,0.0
Minneapolis,20730.55,19556.0
Billings,19869.91,15995.0
Omaha,19565.51,17995.0
Jacksonville,19436.92,1000.0


In [827]:
city_make.tail(2).T.sort_values(['Mean'], ascending=False).head()

,Mean,Median
Albuquerque,100564.90,0.0
Seattle,37128.44,27999.0
Minneapolis,30884.73,17000.0
Anchorage,30145.76,29000.0
Jacksonville,24278.28,16977.0


#### A. At least for this particular maker... it would be timeconsuming. Adding the maker for the current search, just increased the mean and median overall.

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?

#### A. The model year and the odometer


## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).

In [935]:
out = pd.read_csv('/users/manuel/Desktop/dsi-sf-7-materials_manuel/craiglist/craiglist/spiders/out.csv', delimiter='delimiter')


/Users/manuel/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [936]:
out.head()

,"city,price,title"
0,"chicago,750,RV Refrigerator. Like New"
1,"chicago,750,Outside storage for RV's.Inside av..."
2,"chicago,60,Outside storage for RV's.Inside ava..."
3,"chicago,60,Outside storage for RV's.Inside ava..."
4,"chicago,12500,""Dicor RV EPDM roof coating, 1 g..."


In [937]:
out_fixed = out['city,price,title'].apply(lambda x: pd.Series(str(x).split(',', 2)))

In [938]:
out_fixed.rename(columns={0:'City',1:'Price',2:'Title'},inplace=True)

In [939]:
out_fixed.City.unique()

array(['chicago', 'city', 'portland', 'detroit metro', 'houston',
       'seattle', 'philadelphia', 'charlotte', '"jacksonville',
       'albuquerque', 'louisville', 'milwaukee', 'baltimore', 'omaha',
       'minneapolis', 'wichita', 'anchorage', 'charleston', 'fargo',
       'billings'], dtype=object)

In [940]:
out_fixed.groupby(by='City')['Title'].count()

City
"jacksonville    1270
albuquerque      1160
anchorage        1042
baltimore         606
billings          586
charleston        770
charlotte        1390
chicago          2225
city                2
detroit metro    1745
fargo             655
houston          2379
louisville        808
milwaukee         838
minneapolis      2400
omaha             958
philadelphia      884
portland         2400
seattle          2400
wichita           523
Name: Title, dtype: int64


## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._

#### A. Already have multiple pages for multiple citys.